# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [6]:
w = data[data.race=='w']
b = data[data.race=='b']

In [7]:
black_callbacks = data.call[data.race == 'b']
white_callbacks = data.call[data.race == 'w']

In [11]:
# Black Sounding Names Variables

# Sample Size:
sample_black = len(data[data.race == 'b'])

# Number of callbacks for black-sounding names:
number_black_calls = black_callbacks.sum()

print('Black-Sounding Résumés:', sample_black)
print('Callbacks for Black-Sounding Names:', number_black_calls)


print('\n')


# Set up variables for statistics related to Whtie-Sounding names - 

sample_white = len(data[data.race == 'w'])

# Number of callbacks for white-sounding names:
number_white_calls = white_callbacks.sum()

print('White-Sounding Résumés:', sample_white)
print('Callbacks for White-Sounding Names:', number_white_calls)

Black-Sounding Résumés: 2435
Callbacks for Black-Sounding Names: 157.0


White-Sounding Résumés: 2435
Callbacks for White-Sounding Names: 235.0


In [14]:
from scipy.stats import norm

def two_proportion_standard_error(hits1, attempts1, hits2, attempts2):
    proportion1 = hits1/attempts1
    proportion2 = hits2/attempts2
    
    #standard error:
    SE = np.sqrt(proportion1*(1 - proportion1)/attempts1  +  proportion2*(1 - proportion2)/attempts2)
    return SE

def two_proportion_confidence_interval(hits1, attempts1, hits2, attempts2, alpha=0.05):    
    proportion1 = hits1/attempts1
    proportion2 = hits2/attempts2
    difference_of_proportions = proportion1 - proportion2
    
    SE = two_proportion_standard_error(hits1, attempts1, hits2, attempts2)
    
    z_critical_val = norm.ppf(1 - 0.5*alpha)
    
    # Calculate margin of error:
    moe = z_critical_val * SE
    
    # Calculate confidence intervals:
    confidence_lower = proportion2 - proportion1 - moe
    confidence_higher = proportion2 - proportion1 + moe
    
    return difference_of_proportions, moe, confidence_lower, confidence_higher

In [15]:
difference_of_proportions, moe, confidence_lower, confidence_higher = two_proportion_confidence_interval(number_white_calls, sample_white, number_black_calls, sample_black)

print('Difference of Proportions: {0:0.4f}'.format(difference_of_proportions))
print('Margin of Error: {0:0.4f}'.format(moe))
print('95% Confidence Interval: [ {0:0.4f}  {1:0.4f} ]'.format(confidence_lower, confidence_higher))

Difference of Proportions: 0.0320
Margin of Error: 0.0153
95% Confidence Interval: [ -0.0473  -0.0168 ]


In [16]:
def two_proportion_z_test(hits1, attempts1, hits2, attempts2, one_tailed=False):
    proportion1 = hits1/attempts1
    proportion2 = hits2/attempts2
    proportion = (hits1 + hits2)/(attempts1 + attempts2)
    SE = np.sqrt(proportion * (1-proportion) * (1/attempts1 + 1/attempts2))
    
    z = (proportion1 - proportion2)/SE
    p = 1 - stats.norm.cdf(abs(z))
    
    # If one_tailed = False(0), then p *= 2 is calculated:
    p *= 2 - one_tailed
    
    return z, p

### Z and T testing

In [17]:
z, p = two_proportion_z_test(number_white_calls, sample_white, number_black_calls, sample_black)

print('z-statistic: {0:0.4f}'.format(z))
print('p-value: {0:0.4f}'.format(p))

z-statistic: 4.1084
p-value: 0.0000


With the findings above we see that the p value is less than 0.05 and the confidence interval does not have zero, hence the null hypothesis is rejected.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

Race is indeed one feature in receiving interview callbacks. Other factors have not been tested in a similar fashion. Gender could be a factor.